# Building tests

In [ ]:
from sshdol import SshFiles
from functools import partial 

# make a bytes files ssh store
s = SshFiles(host=SSH_TEST_HOST, rootdir=SSH_TEST_ROOTDIR)
empty_test_store(s)

assert list(s) == []  # s is empty

s['apple'] = b'crumble'  # make an apple file containing b'crumble'
assert 'apple' in s  # apple is now in s
assert list(s) == ['apple']  # now, s has only one file, apple
assert s['apple'] == b'crumble'  # see that the value of s['apple'] is indeed b'crumble'

# make a text files ssh store
s = SshFiles(encoding='utf-8', host=SSH_TEST_HOST, rootdir=SSH_TEST_ROOTDIR)

s['apple.txt'] = 'sauce'  # make an apple file containing 'sauce'
assert 'apple.txt' in s  # apple is now in s
assert sorted(s) == ['apple', 'apple.txt']  # now, s has only one file, apple
assert s['apple'] == 'sauce'  # see that the value of s['apple'] is indeed 'sauce'

In [3]:
from sshdol import SshFiles
import pytest
from sshdol.tests.utils_for_testing import empty_test_store, SSH_TEST_HOST, SSH_TEST_ROOTDIR


s = SshFiles(
    host=SSH_TEST_HOST,
    rootdir=SSH_TEST_ROOTDIR,
    max_levels=None,
    create_dirs=True,
)
empty_test_store(s)

# Test recursive write with directory creation (unlimited depth)
s['level0/level1/level2/file.txt'] = b'deep file content'

# Test recursive read
assert s['level0/level1/level2/file.txt'] == b'deep file content'

# Test recursive iteration (with unlimited depth)
all_keys = list(s)
assert 'level0/' in all_keys
assert 'level0/level1/' in all_keys
assert 'level0/level1/level2/' in all_keys
assert 'level0/level1/level2/file.txt' in all_keys

# Test with limited recursion depth
s_limited = SshFiles(
    host=SSH_TEST_HOST,
    rootdir=SSH_TEST_ROOTDIR,
    max_levels=1,  # Only one level of subdirectories
    create_dirs=True,
)

# Should only see first level directories and files
limited_keys = list(s_limited)
assert 'level0/' in limited_keys
assert 'level0/level1/' in limited_keys
assert 'level0/level1/level2/' not in limited_keys

# Writing at allowed depth should work
s_limited['level0/file.txt'] = b'allowed depth content'
assert s_limited['level0/file.txt'] == b'allowed depth content'

# Test that reading beyond max_levels raises KeyError with appropriate message
with pytest.raises(KeyError) as excinfo:
    _ = s_limited['level0/level1/level2/file.txt']
assert "Path depth (3) exceeds maximum allowed depth (1)" in str(excinfo.value)

# Test that writing beyond max_levels raises KeyError with appropriate message
with pytest.raises(KeyError) as excinfo:
    s_limited['level0/level1/file.txt'] = b'beyond max depth'
assert "Path depth (2) exceeds maximum allowed depth (1)" in str(excinfo.value)

# Test with level 0 (only root level)
s_root_only = SshFiles(
    host=SSH_TEST_HOST,
    rootdir=SSH_TEST_ROOTDIR,
    max_levels=0,
)

# Should be able to access files in the root
root_keys = list(s_root_only)
assert 'level0/' in root_keys
assert 'level0/level1/' not in root_keys

# Test that accessing even one level deep raises error
with pytest.raises(KeyError) as excinfo:
    _ = s_root_only['level0/file.txt']
assert "Path depth (1) exceeds maximum allowed depth (0)" in str(excinfo.value)

# Test membership with recursion
assert 'level0/level1/level2/file.txt' in s  # Unlimited depth store
assert 'level0/level1/level2/nonexistent.txt' not in s



In [4]:

# Test membership check also respects max_levels

assert 'level0/level1/file.txt' not in s_limited

# Test length with recursion
# Length should include all files and directories up to max_levels
assert len(s) > len(s_limited)


In [8]:
'level0/level1/file.txt' in s_limited

False

In [12]:
list(s)

['level0/',
 'level0/file.txt',
 'level0/level1/',
 'level0/level1/level2/',
 'level0/level1/level2/file.txt']

In [10]:
s_limited._rootdir

'/root/data/tests/sshdol'

In [5]:
root_keys = list(s_root_only)
root_keys

['level0/']

In [ ]:
from sshdol import SshFiles

from sshdol.tests.utils_for_testing import empty_test_store, SSH_TEST_HOST, SSH_TEST_ROOTDIR


s = SshFiles(
    host=SSH_TEST_HOST,
    rootdir=SSH_TEST_ROOTDIR,
    max_levels=None,
    create_dirs=True,
)
empty_test_store(s)

# Test recursive write with directory creation
s['level0/level1/level2/file.txt'] = b'deep file content'


In [2]:
s_limited = SshFiles(
    host=SSH_TEST_HOST,
    rootdir=SSH_TEST_ROOTDIR,
    max_levels=1,  # Only one level of subdirectories
)

s_limited['level0/level1/level2/file.txt']

KeyError: 'Path depth (3) exceeds maximum allowed depth (1): level0/level1/level2/file.txt'

In [12]:
from sshdol import SshFiles

from sshdol.tests.utils_for_testing import empty_test_store, SSH_TEST_HOST, SSH_TEST_ROOTDIR


s = SshFiles(
    host=SSH_TEST_HOST,
    rootdir=SSH_TEST_ROOTDIR,
    max_levels=None,
    create_dirs=True,
)
empty_test_store(s)

# Test recursive write with directory creation
s['level0/level1/level2/file.txt'] = b'deep file content'

# Test recursive read
assert s['level0/level1/level2/file.txt'] == b'deep file content'

# Test recursive iteration (with unlimited depth)
all_keys = list(s)
assert 'level0/' in all_keys
assert 'level0/level1/' in all_keys
assert 'level0/level1/level2/' in all_keys
assert 'level0/level1/level2/file.txt' in all_keys

# Test with limited recursion depth
s_limited = SshFiles(
    host=SSH_TEST_HOST,
    rootdir=SSH_TEST_ROOTDIR,
    max_levels=1,  # Only one level of subdirectories
)

# Should only see first level directories and files
limited_keys = list(s_limited)
assert 'level0/' in limited_keys
assert 'level0/level1/' in limited_keys
assert 'level0/level1/level2/' not in limited_keys

# But we can still access deeper paths directly
assert s_limited['level0/level1/level2/file.txt'] == b'deep file content'

# Test membership with recursion
assert 'level0/level1/level2/file.txt' in s
assert 'level0/level1/level2/nonexistent.txt' not in s

# Test length with recursion
# Length should include all files and directories up to max_levels
assert len(s) > len(s_limited)


In [13]:
s_limited['level0/level1/level2/another_file.txt'] = b'another file content'

In [14]:
s_limited['level0/level1/level2/another_file.txt']

b'another file content'

In [8]:

# Test recursive iteration (with unlimited depth)
all_keys = list(s)
assert 'level1/' in all_keys
assert 'level1/level2/' in all_keys
assert 'level1/level2/level3/' in all_keys
assert 'level1/level2/level3/file.txt' in all_keys


In [9]:

# Test with limited recursion depth
s_limited = SshFiles(
    host=SSH_TEST_HOST,
    rootdir=SSH_TEST_ROOTDIR,
    max_levels=1,  # Only one level of subdirectories
)


In [ ]:
# Should only see first level directories and files
limited_keys = list(s_limited)
assert 'level1/' in limited_keys
assert 'level1/level2/' in limited_keys
assert 'level1/level2/level3' in limited_keys


AssertionError: 

In [11]:
limited_keys

['level1/', 'level1/level2/']

In [ ]:

# # But we can still access deeper paths directly
# assert s_limited['level1/level2/level3/file.txt'] == b'deep file content'

# # Test membership with recursion
# assert 'level1/level2/level3/file.txt' in s
# assert 'level1/level2/level3/nonexistent.txt' not in s

# # Test length with recursion
# # Length should include all files and directories up to max_levels
# assert len(s) > len(s_limited)


In [ ]:
from sshdol import SshFiles
from dol import TextFiles

s = SshFiles(host=SSH_TEST_HOST, rootdir=SSH_TEST_ROOTDIR, encoding='utf-8')

empty_test_store(s)

assert list(s) == []  # s is empty
s['apple.txt'] = 'sauce'  # make an apple file containing 'sauce'
assert 'apple.txt' in s  # apple is now in s
assert sorted(s) == ['apple', 'apple.txt']  # now, s has only one file, apple
assert s['apple'] == 'sauce'  # see that the value of s['apple'] is indeed 'sauce'

# Getting a bunch of data from a server

In [ ]:
from sshdol import SshFiles

s = SshFiles(host="thorwhalen", rootdir="/root/.config/mood/news/searches")


In [ ]:
list(s)

['newsdata/', 'yahoo_finance_headlines/', 'yahoo_finance/']

In [ ]:
newsdata = s['newsdata/']
list(newsdata)

['2025-03-03/',
 '2025-03-02/',
 '2025-02-20/',
 '2025-03-01/',
 '2025-02-19/',
 '2025-02-25/',
 '2025-03-04/',
 '2025-03-05/',
 '2025-03-08/',
 '2025-03-07/',
 '2025-03-09/',
 '2025-03-06/',
 '2025-02-24/',
 '2025-02-26/',
 '2025-02-21/',
 '2025-02-28/',
 '2025-02-27/',
 '2025-03-10/',
 '2025-02-22/',
 '2025-02-23/']

In [ ]:
from sshdol import SshFiles

s = SshFiles(host="thorwhalen")

list(s)

['misc.ipynb', 'data/', 'py/', 'scripts/']

In [ ]:
sub_s_under_data_subfolder = s['data/']
sub_s_under_data_subfolder._rootdir

'data'

In [ ]:
list(sub_s_under_data_subfolder)

['financial_news_search_words.json', 'scoopy/']

In [ ]:
another_sub_folder = sub_s_under_data_subfolder['scoopy/']
another_sub_folder._rootdir

'data/scoopy'

In [ ]:
list(another_sub_folder)

['2025-02-18--22-37-40__Recession fears.json',
 '2025-02-18--22-37-26__Interest rates.json',
 '2025-02-18--22-37-30__Unemployment rate.json',
 '2025-02-18--22-37-38__Election results.json',
 '2025-02-18--22-37-44__Risk appetite.json',
 '2025-02-18--22-37-31__Consumer confidence.json',
 '2025-02-18--22-37-29__Earnings report.json',
 '2025-02-18--22-37-36__Trade war.json',
 '2025-02-18--22-37-38__Brexit.json',
 '2025-02-18--22-37-27__Inflation.json',
 '2025-02-18--22-37-41__Black Swan event.json',
 '2025-02-18--22-37-34__Stock market rally.json',
 '2025-02-18--22-37-37__Sanctions.json',
 '2025-02-18--22-37-34__Market correction.json',
 '2025-02-18--22-30-51__Interest rates.json',
 '2025-02-18--22-37-40__Market volatility.json',
 '2025-02-18--22-37-28__Federal Reserve.json',
 '2025-02-18--22-37-35__Oil prices.json',
 '2025-02-18--22-37-39__Geopolitical tensions.json',
 '2025-02-18--22-37-30__GDP growth.json']

In [ ]:
sub_s_under_data_subfolder

'data/scoopy/'